In [ ]:
import lightgbm as lgbm
from matplotlib import pyplot as plt
from matplotlib.lines import Line2D
import seaborn as sns
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
from gamexplainer import GamExplainer
from collections import defaultdict
from sklearn.model_selection import GridSearchCV
from math import comb
import pickle
from sklearn.metrics import mean_squared_error
%load_ext autoreload
%autoreload 2

In [ ]:
plt.rcParams['text.usetex'] = False

In [ ]:
!wget -O superconduct.zip https://archive.ics.uci.edu/ml/machine-learning-databases/00464/superconduct.zip

In [ ]:
!unzip superconduct.zip

## Read the dataset

In [ ]:
df = pd.read_csv("train.csv", sep=",")

In [ ]:
df

In [ ]:
df = pd.read_csv("train.csv", sep=",")
train = df.head(int(len(df) * 0.7))
test = df.tail(len(df) - len(train))
resp_var = "critical_temp"
X_train = train.drop("critical_temp", axis=1)
y_train = train[resp_var]
X_test = test.drop(resp_var, axis=1)
y_test = test[resp_var]

## Train the forest

In [ ]:
lgbm_info = {}
parameters = {
    "n_estimators": np.geomspace(100, 10000, num=3, dtype=int),
    "num_leaves": np.geomspace(32, 256, num=4, dtype=int),
    "learning_rate": np.geomspace(1e-3, 1e-1, num=3)
}
CV_regressor = GridSearchCV(lgbm.LGBMRegressor(n_jobs=16), parameters, verbose=3, scoring="neg_root_mean_squared_error")
CV_regressor.fit(X_train, y_train)

In [ ]:
print(CV_regressor.best_params_)

Best params:
{'learning_rate': 0.001, 'n_estimators': 10000, 'num_leaves': 64}

In [ ]:
forest = lgbm.LGBMRegressor(learning_rate=0.001, n_estimators=10000, num_leaves=64, n_jobs=16, verbose=2)
forest.fit(X_train, y_train)

In [ ]:
print(mean_squared_error(y_test, forest.predict(X_test), squared=False))

In [ ]:
np.max(y_train)

## Feature selection

In [ ]:
range_n_splines = range(1, 11)
range_n_inter = range(0, 9)

### Create the numpy array

In [ ]:
explanation_params = {"verbose": False,
                      "sample_method": "all",
                      "inter_max_distance": 64}

acc = np.zeros((len(range_n_splines), len(range_n_inter)))
for i, n_splines in enumerate(tqdm(range_n_splines)):
    explanation_params["n_spline_terms"] = n_splines
    for j, n_inter in enumerate(range_n_inter):
        if n_inter > comb(n_splines, 2):
            continue
        explanation_params["n_inter_terms"] = n_inter
        explainer = GamExplainer(**explanation_params)
        gam = explainer.explain(forest, lam_search_space=[0.1, 1])

        acc[i, j] = explainer.loss_res

In [ ]:
acc

In [ ]:
np.save("feat_selection_superconduct", acc)

### Or load it if already saved

In [ ]:
acc = np.load("feat_selection_superconduct.npy")

## Plot the results in a heatmap

In [ ]:
dimension = (len(range_n_splines), len(range_n_inter))
mask = np.zeros(dimension)
for i, n_splines in enumerate(tqdm(range_n_splines)):
    for j, n_inter in enumerate(range_n_inter):
        if n_inter > comb(n_splines, 2):
            mask[i, j] = True 
            continue

In [ ]:
accuracy_df = pd.DataFrame(acc, columns=range_n_inter, index=range_n_splines)
ax = sns.heatmap(accuracy_df, annot=True, mask = mask, cmap=sns.color_palette("Blues", as_cmap=True), cbar_kws={'label': 'RMSE'})
ax.set_xlabel("Number of interaction terms used")
ax.set_ylabel("Number of splines used")
file_out = "splines_inter.pdf"
plt.savefig(file_out)

## Sampling strategy 

### Analyze the maximum number of splits per feature

In [ ]:
explanation_params = {"verbose": False,
                      "interaction_importance_method":"count_path",
                      "feat_importance_method": "gain",
                      "n_spline_terms": 7,
                      "sample_method": "all",
                      "n_spline_per_term": 50,
                      "inter_max_distance": 64,
                      "n_inter_terms": 0,
                      "n_sample_gam":int(1e5),
                      "portion_sample_test":0.3,
                      "classification": False
                      }
explainer = GamExplainer(**explanation_params)
gam = explainer.explain(forest, lam_search_space=[0.1, 1])

In [ ]:
for key, value in explainer.feature_dict.items():
    if key in explainer.mif:
        print(f"{key}: {len(value)}")

In [ ]:
sampling_methods = ["all", "quantile", "equal", "kmeans", "equi_size"]
range_m = range(50, 17000, 750)

### Load it

In [ ]:
with open('sampling_comparison.pickle', 'rb') as f:
    acc_methods = pickle.load(f)

### Or compute it

In [ ]:
explanation_params = {"verbose": False,
                      "interaction_importance_method":"count_path",
                      "feat_importance_method": "gain",
                      "n_spline_terms": 7,
                      "n_spline_per_term": 50,
                      "inter_max_distance": 64,
                      "n_inter_terms": 0,
                      "n_sample_gam":int(1e5),
                      "portion_sample_test":0.3,
                      "classification": False
                      }
acc_methods = defaultdict(list)
for m in tqdm(range_m):
    explanation_params["sample_n"] = m
    for sampling_method in sampling_methods:
        explanation_params["sample_method"] = sampling_method
        explainer = GamExplainer(**explanation_params)
        gam = explainer.explain(forest, lam_search_space=[0.1, 1])

        acc_methods[sampling_method].append(explainer.loss_res)

In [ ]:
with open('sampling_comparison.pickle', 'wb') as f:
    pickle.dump(acc_methods, f)

### Plot

In [ ]:
labels = [r"\emph{All-Thresholds}", r"\emph{Quantile}", r"\emph{Equi-Width}", r"\emph{$k$-Means}", "\emph{Equi-Size}"]
colors = sns.color_palette(n_colors=len(sampling_methods))
for i, sampling_method in enumerate(sampling_methods):
    plt.plot(range_m, acc_methods[sampling_method], 'o--', color=colors[i], label=labels[i])
plt.xlabel("$K$")
plt.ylabel("RMSE")
plt.legend()
file_out = "sampling_comparison.pdf"
plt.savefig(file_out)

## Splines investigation

In [ ]:
explanation_params = {
                      "verbose": False,
                      "interaction_importance_method":"count_path",
                      "feat_importance_method": "gain",
                      "n_spline_terms": 7,
                      "sample_method": "equi_size",
                      "sample_n": 4500,
                      "n_spline_per_term": 50,
                      "inter_max_distance": 64,
                      "n_inter_terms": 0,
                      "n_sample_gam":int(1e5),
                      "portion_sample_test":0.3,
                      "classification": False
                      }
explainer = GamExplainer(**explanation_params)
explainer.explain(forest)

## With sample highlighting

In [ ]:
feature_names_display = {i: feat for i, feat in enumerate(X_train.columns)}

In [ ]:
X_train.columns[6]

In [ ]:
feature_names_display[6] = "WEAM"
feature_names_display[62] = "WMTC"
feature_names_display[70] = "WSTC"
feature_names_display[76] = "WEV"
feature_names_display[74] = "WGV"
feature_names_display[9] = "SAM"
feature_names_display[33] = "GMD"
feature_names_display[64] = "WGTC"
feature_names_display[44] = "WGEA"

In [ ]:
sample_index = 0
sample = X_train.iloc[sample_index].values.reshape(1, -1)

In [ ]:
n_row, n_col = 2, 3

fig = plt.figure(figsize=(13, 10), tight_layout=False)

lines = []

terms = [(i, x) for i, x in enumerate(explainer.gam.terms) if not x.isintercept and not x.istensor]
terms.sort(key=lambda x: x[1].feature)
c1, c2, c3 = sns.color_palette(n_colors=3)

plot_index = 0
axes = []
for i, term in enumerate(explainer.gam.terms):
    if i == 6:
        break
    if term.isintercept or term.istensor:
        continue
    
    ax = fig.add_subplot(n_row, n_col, plot_index + 1, sharey = axes[-1] if plot_index % n_col != 0 else None)

    plt.setp(ax.get_yticklabels(), visible=plot_index % n_col == 0)
    
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
        
    
    # Spline print
    grid = explainer.gam.generate_X_grid(term=i, meshgrid=term.istensor)
    pdep, confi = explainer.gam.partial_dependence(term=i, X=grid, width=0.95, meshgrid=term.istensor)
  
    conf_u = ax.plot(grid[:, term.feature], confi[:,0], ls="--", c=c2, zorder=1)
    conf_l = ax.plot(grid[:, term.feature], confi[:,1], label="95% width confidence interval", ls="--", c=c2, zorder=1)
    l1 = ax.plot(grid[:, term.feature], pdep, label="Spline learned", lw=2, c=c1, zorder=2)
    ax.set_title(feature_names_display[term.feature])
    
    
    # Print the sample
    x_point = sample[0, term.feature] # col vector
    y_point = explainer.gam.partial_dependence(term=i, X=sample)
    
    plt.vlines(x_point, ax.get_ylim()[0], y_point, linestyle="dashed", color="black")
    plt.hlines(y_point, ax.get_xlim()[0], x_point, linestyle="dashed", color="black")
    ax.scatter(x_point, y_point, label="Sample under investigation", color="black", zorder=3)
    
    
    plot_index +=1
    axes.append(ax)


#plt.subplots_adjust(hspace=0.3)
file_out = "generators.pdf"

params = {'legend.fontsize': 18,
          'figure.figsize': (20, 5),
          'axes.titlesize': 18,
          'xtick.labelsize': 18,
          'ytick.labelsize': 18}
plt.rcParams.update(params)

file_out = "generators.pdf"
plt.legend(loc='upper center', bbox_to_anchor=(-0.7, 2.5), ncol=3)
plt.savefig(file_out)

## Results with SHAP

### Load them

In [ ]:
with open('shapley_values_training.pickle', 'rb') as f:
    shap_values = pickle.load(f)
with open('shap_explainer_training.pickle', 'rb') as f:
    shap_explainer = pickle.load(f)

### Or compute them

In [ ]:
import shap

shap_explainer = shap.Explainer(forest)
shap_values = shap_explainer(X_train)


In [ ]:
with open('shapley_values_training.pickle', 'wb') as f:
    pickle.dump(shap_values, f)
with open('shap_explainer_training.pickle', 'wb') as f:
    pickle.dump(shap_explainer, f)

In [ ]:
shap.plots.beeswarm(shap_values)

In [ ]:
# visualize the first prediction's explanation
shap.plots.force(shap_explainer.expected_value, shap_values.values[0,:], matplotlib=True)

In [ ]:
shap_values = shap_values[:, :]

In [ ]:
n_row, n_col = 2, 3

fig = plt.figure(figsize=(13, 10))

lines = []

terms = [(i, x) for i, x in enumerate(explainer.gam.terms) if not x.isintercept and not x.istensor]
terms.sort(key=lambda x: x[1].feature)
c1, c2, c3 = sns.color_palette(n_colors=3)

plot_index = 0
axes = []
for i, term in enumerate(explainer.gam.terms):
    if i == 6:
        break
    if term.isintercept or term.istensor:
        continue
    
    ax = fig.add_subplot(n_row, n_col, plot_index + 1, sharey = axes[-1] if plot_index % n_col != 0 else None)
    
    # Shap scatter print
    shap.plots.scatter(shap_values[:,term.feature], ax=ax, show=False, hist=False, color=c1)
    shap_plot = ax
    
    plt.setp(ax.get_yticklabels(), visible=plot_index % n_col == 0)
    
    ax.set_ylabel("")
    ax.set_xlabel("")
    ax.tick_params(labelsize=18)
    ax.set_title(feature_names_display[term.feature])
    
    # Print the sample
    x_point = shap_values[sample_index, term.feature].data
    y_point = shap_values[sample_index, term.feature].values
    
    plt.vlines(x_point, ax.get_ylim()[0], y_point, linestyle="dashed", color="black")
    plt.hlines(y_point, ax.get_xlim()[0], x_point, linestyle="dashed", color="black")
    sample_plot = ax.scatter(x_point, y_point, label="Sample under investigation", color="black", zorder=3)

    plot_index +=1
    axes.append(ax)

params = {'legend.fontsize': 18,
          'figure.figsize': (20, 5),
          'axes.titlesize': 18}

plt.rcParams.update(params)
#plt.subplots_adjust(hspace=0.3)
file_out = "shap.pdf"
dummy_shap_plot = Line2D([0], [0], marker='o', color=c1, label='SHAP values', lw=0)
plt.legend(handles = [dummy_shap_plot, sample_plot], loc='upper center', bbox_to_anchor=(-1.47, 2.5), ncol=3, fontsize=14)
plt.savefig(file_out)

## Local explanation

In [ ]:
new_feature_names = [feature_names_display.get(i, feat) for i, feat in enumerate(X_train.columns)]
shap_values.feature_names = new_feature_names

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
shap.plots.waterfall(shap_values[sample_index], max_display=7, show=False)
plt.rcParams.update({'font.size': 14})
plt.tight_layout()
file_out = "local_explain_shap.pdf"
plt.savefig(file_out)

In [ ]:
from gamexplainer.utils import plot_local_all_terms
%load_ext autoreload
%autoreload 2

In [ ]:
plot_local_all_terms(explainer.gam, feature_names_display, X_train.values, sample_index, range_perc = 20, figsize=(9, 15))
file_out = "local_explain_gef.pdf"
plt.savefig(file_out)